In [1]:
import pandas as pd

books = pd.read_csv("data/books.csv")
ratings = pd.read_csv("data/ratings.csv")

## Collaborative Filtering by model (SVD)

In [2]:
import numpy as np
from scipy.sparse.linalg import svds
from scipy.linalg import sqrtm

# Création de la table user // book avec les notes déjà données
R_df = ratings.pivot(index="user_id",columns="book_id",values="rating").fillna(0)

# on récupère seulement les valeurs pour le svds
R = R_df.values

# Calcul la moyenne des notes mise par chaque user, pour la retirer (on la rajoutera à la fin)
user_ratings_mean = np.mean(R, axis=1)
user_ratings_mean = np.float16(user_ratings_mean)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

latent_dimension = 90 # Après une étude sur cette valeur (de 90 à 110) et sur l'erreur myenne et écart type prediction - note réelle, on a décidé de prendre 90 (rapport erreur/temps de calcul)

U, sigma, Vt = svds(R_demeaned, k=latent_dimension) 

sigma = np.diag(sigma)
s_root = sqrtm(sigma)

Usk = np.dot(U, s_root)
Usk = np.float16(Usk) # Passage float64 à float16

skV = np.dot(s_root, Vt)
skV = np.float16(skV) # Passage float64 à float16

predicted_rating = np.dot(Usk, skV) # Calcul des notes de chaque user pour chaque livre

predicted_rating = predicted_rating + user_ratings_mean.reshape(-1, 1) # On rajoute la moyenne

preds_df = pd.DataFrame(predicted_rating, columns=R_df.columns, index=R_df.index)

In [10]:
def get_book_name(book_id):
    # Fonction qui récupère le titre du livre grâce à son id
    book_name = books["title"][books["book_id"]==book_id].item()
    return book_name

def recommandation(user, preds_df=preds_df):
    # Fonction qui, pour un utilisateur, renvoie la liste de ses 10 livres préférés (meilleures notes) ainsi que les 10 livres qui lui sont le plus recommandés

    list_recommandations = []

    # Récupère la ligne qui correspond à l'user dans les notes originales et les notes prédites
    original_ratings = R_df.loc[user]
    pred_ratings = preds_df.loc[user]

    prefered_books = pred_ratings[original_ratings!=0].sort_values(axis=0, ascending=False)
    pred_ratings = pred_ratings[original_ratings==0].sort_values(axis=0, ascending=False) # On garde les note qui n'étaient pas présentes dans les notes originales et on les trie

    # On récupère les indices (= id) des 10 premiers 
    list_prefered = list(prefered_books[0:10].index)
    list_recommandations = list(pred_ratings[0:10].index)

    # On récupère leurs noms
    list_prefered = list(map(get_book_name,list_prefered))
    list_recommandations = list(map(get_book_name,list_recommandations))

    return list_prefered, list_recommandations

for i in range(1,10):
    list_prefered, list_recommandations = recommandation(user=i)
    print("User n°",i)
    print("Livres préférés :")
    print(list_prefered)

    print("\nLivres recommandés :")
    print(list_recommandations)
    print("_______________\n")


User n° 1
Livres préférés :
['To Kill a Mockingbird', 'The Kite Runner', 'Memoirs of a Geisha', 'Pride and Prejudice', 'The Girl with the Dragon Tattoo (Millennium, #1)', 'A Thousand Splendid Suns', 'The Help', 'Life of Pi', "Ender's Game (Ender's Saga, #1)", 'The Alchemist']

Livres recommandés :
['Anna Karenina', 'The Little Prince', 'Wuthering Heights', 'The Guernsey Literary and Potato Peel Pie Society', 'Cutting for Stone', 'A Tree Grows in Brooklyn', 'The Joy Luck Club', 'The Other Boleyn Girl (The Plantagenet and Tudor Novels, #9)', "My Sister's Keeper", 'The Road']
_______________

User n° 2
Livres préférés :
['Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)', 'Harry Potter and the Goblet of Fire (Harry Potter, #4)', 'Harry Potter and the Half-Blood Prince (Harry Potter, #6)', 'The Great Gatsby', 'Harry Potter and the Deathly Hallows (Harry Potter, #7)', 'Harry Potter and the Chamber of Secrets (Harry Potter, #2)', 'Harry Potter and the Order of the Phoenix (Harry P